Tutorial followed from: https://www.youtube.com/watch?v=yqkISICHH-U&ab_channel=NicholasRenotte
Source code found here: https://github.com/nicknochnack/TFODCourse

Main changes were the larger dataset for more classified signs, created by our group members.

In [ ]:
import os

CUSTOM_MODEL_NAME = 'family_v1'  # change this to test differnet model
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'


paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}


In [ ]:
## Making a lable map

labels = [{'name':'step', 'id':1}, 
          {'name':'mother', 'id':2}, 
          {'name':'father', 'id':3}, 
          {'name':'son', 'id':4}, 
          {'name':'daughter', 'id':5}, 
          {'name':'brother', 'id':6}, 
          {'name':'sister', 'id':7}, 
          {'name':'baby', 'id':8},
          {'name':'home', 'id':9},
          {'name':'my', 'id':10},
          {'name':'your', 'id':11}]

# labels = [{'name':'hello', 'id':1}, 
#           {'name':'thanks', 'id':2}, 
#           {'name':'name', 'id':3}, 
#           {'name':'good', 'id':4}, 
#           {'name':'luck', 'id':5}, 
#           {'name':'bad', 'id':6}, 
#           {'name':'meet', 'id':7}, 
#           {'name':'you', 'id':8},
#           {'name':'morning', 'id':9},
#           {'name':'afternoon', 'id':10},
#           {'name':'how', 'id':11}]



with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)

checkpoint = 'ckpt-7' # Change this to the checkpoint / snapshot of the model to test (check for early stopping benefits)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], checkpoint)).expect_partial()

@tf.function
def detect_fn(image): # This function is used to detect the objects in the image
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [ ]:
import cv2 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
## Test model prediction from one image for debugging purposes

IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'hello.898202b9-b153-11ed-ac55-5c879c62795a.jpg')


img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np_with_detections,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=5,
            min_score_thresh=.58,
            agnostic_mode=False)

plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
## Automated testing script

import os
import tensorflow as tf
from object_detection.utils import visualization_utils as viz_utils
import cv2 
import numpy as np
from matplotlib import pyplot as plt # Used to plot the images if required

# signs = ['hello', 'thanks', 'name', 'good', 'luck', 'bad', 'meet', 'you', 'morning', 'afternoon', 'how'] # List of signs for GREETINGS category
signs = ['step', 'mother', 'father', 'son', 'daughter', 'brother', 'sister', 'baby', 'home', 'my', 'your'] # List of signs for FAMILY category
folder_path = 'Tensorflow/workspace/images/family_dataset/external_test' # Path to test images directory
correct_counts = {} # Dictionary to store the correct count for each sign

# Loop over each file in the folder and run them trhough the model one by one, checking prediction against the sign in the file name
for file_name in os.listdir(folder_path):
    if file_name.endswith(".jpg"):
        img_path = os.path.join(folder_path, file_name) # Path to the image
        img = cv2.imread(img_path) # Load the image
        image_np = np.array(img) # Convert the image to a numpy array
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32) # Convert the image to a tensor
        detections = detect_fn(input_tensor) # Run the detection function on the image
            
        # Get the predicted class and score
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
        detections['num_detections'] = num_detections
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        viz_utils.visualize_boxes_and_labels_on_image_array(
                    image_np_with_detections,
                    detections['detection_boxes'],
                    detections['detection_classes']+label_id_offset,
                    detections['detection_scores'],
                    category_index,
                    use_normalized_coordinates=True,
                    max_boxes_to_draw=5,
                    min_score_thresh=.4,
                    agnostic_mode=False)
        prediction = ''
        expected_label = file_name.split(".")[0] # Determine the expected label based on the filename
        # Determine the model prediction
        for i in range(len(detections['detection_classes'])):
            if detections['detection_scores'][i] >= 0.4:
                prediction = signs[detections['detection_classes'][i]]
                
        if expected_label not in correct_counts:
            correct_counts[expected_label] = 0 # Initialize the correct count for this label to 0 if it doesn't exist yet
        if prediction == expected_label: # Compare the predicted label to the expected label
            if expected_label in correct_counts:
                correct_counts[expected_label] += 1 # Increment the correct count for this label if the prediction was correct

    ##### Uncomment this to see the images with the bounding boxes
    plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.show()
print("Testing model: " + CUSTOM_MODEL_NAME)
print("Testing checkpint: " + checkpoint)
for label, count in correct_counts.items(): # Print the correct count for each label
    print("Correctly identified {} {}/10 times".format(label, count))